In [1]:
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import os
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, train_mlp, test_mlp, train_mlpbeta, test_mlpbeta, train_cnn, test_cnn
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch
set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'AB'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,feat,feat_sq = pickle.load(f)

In [3]:
sess_in = {'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'manual','scaler_load':False,'feat_type':'feat','sub':2}

sess = session.Sess(**sess_in)

ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
if np.sum(ind):
    trainmlp, validmlp, traincnn, validcnn, y_train_clean, y_valid_clean, x_train_noise_mlp, x_train_noise_cnn = prd.prep_train_data(sess,raw,params)

Loading training data: traindata_manual/AB2_traindata_2.p


In [4]:
mlp = MLP()
mlp_beta = MLPbeta()
cnn = CNN()

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [5]:
EPOCHS = 10
models = ['mlp','mlpbeta','cnn']

for model in models:
    print('Training ' + model)
    for epoch in range(EPOCHS):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        # Train MLP
        if model == 'mlp':
            for x, y in trainmlp:
                train_mlp(x, y, mlp, loss_fn, optimizer, train_loss, train_accuracy)
            for x_test, y_test in validmlp:
                test_mlp(x_test, y_test, mlp, loss_fn, test_loss, test_accuracy)
        # Train MLP Beta
        elif model == 'mlpbeta':
            for x, y in trainmlp:
                train_mlpbeta(x, y, mlp_beta, loss_fn, optimizer, train_loss, train_accuracy)
            for x_test, y_test in validmlp:
                test_mlpbeta(x_test, y_test, mlp_beta, loss_fn, test_loss, test_accuracy)
        # Train CNN
        elif model == 'cnn':
            for x, y in traincnn:
                train_cnn(x, y, cnn, loss_fn, optimizer, train_loss, train_accuracy)
            for x_test, y_test in validcnn:
                test_cnn(x_test, y_test, cnn, loss_fn, test_loss, test_accuracy)

        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )

Training mlp
Epoch 1, Loss: 1.9388959407806396, Accuracy: 17.408729553222656, Test Loss: 1.925368309020996, Test Accuracy: 22.69841194152832
Epoch 2, Loss: 1.881583571434021, Accuracy: 25.71825408935547, Test Loss: 1.8604031801223755, Test Accuracy: 27.58730125427246
Epoch 3, Loss: 1.8358728885650635, Accuracy: 30.809524536132812, Test Loss: 1.8282634019851685, Test Accuracy: 31.555553436279297
Epoch 4, Loss: 1.807121992111206, Accuracy: 35.19047546386719, Test Loss: 1.8037445545196533, Test Accuracy: 35.80952453613281
Epoch 5, Loss: 1.7826353311538696, Accuracy: 38.05158615112305, Test Loss: 1.7808690071105957, Test Accuracy: 38.269840240478516
Epoch 6, Loss: 1.7650784254074097, Accuracy: 39.67856979370117, Test Loss: 1.770175814628601, Test Accuracy: 39.07936477661133
Epoch 7, Loss: 1.7516915798187256, Accuracy: 41.130950927734375, Test Loss: 1.7649626731872559, Test Accuracy: 38.93650817871094
Epoch 8, Loss: 1.7392101287841797, Accuracy: 42.52777862548828, Test Loss: 1.7578219175338

In [6]:
mlp_enc = mlp.get_layer(name='enc')
mlpbeta_enc = mlp_beta.get_layer(name='enc')
cnn_enc = cnn.get_layer(name='enc')

mlp_aligned = mlp_enc(x_train_noise_mlp).numpy()
mlp_beta_aligned = mlpbeta_enc(x_train_noise_mlp).numpy()
cnn_aligned = cnn_enc(x_train_noise_cnn).numpy()

y_train_aligned = np.argmax(y_train_clean, axis=1)[...,np.newaxis]
w_mlp, c_mlp,_, _, _ = train_lda(mlp_aligned,y_train_aligned)
w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlp_beta_aligned,y_train_aligned)
w_cnn, c_cnn,_, _, _ = train_lda(cnn_aligned,y_train_aligned)

In [7]:
test_sess = {'test_grp':4, 'test':'partposrealmixeven24'}
sess.update(**test_sess)

with open('real_noise/all_real_noise.p', 'rb') as f:
    real_noise_temp, _ = pickle.load(f)

x_test_cnn, x_test_mlp, y_test_clean, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)

mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
y_test_aligned = np.argmax(y_test_clean, axis=1)[...,np.newaxis]

clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test_clean,mlp,clean_size)
clean_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
noisy_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'MLP ---- '
    f'Clean Accuracy: {clean_acc * 100:.2f}, '
    f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
    f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
    f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
)

clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test_clean,mlp_beta,clean_size)
clean_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
noisy_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'MLPB ---- '
    f'Clean Accuracy: {clean_acc * 100:.2f}, '
    f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
    f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
    f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
)

clean_acc, noisy_acc = eval_nn(x_test_cnn, y_test_clean,cnn,clean_size)
clean_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
noisy_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'CNN ---- '
    f'Clean Accuracy: {clean_acc * 100:.2f}, '
    f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
    f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
    f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
)

Loading training data: traindata_manual/AB2_traindata_4.p
MLP ---- Clean Accuracy: 59.37, Noisy Accuracy: 54.83, LDA Clean Accuracy: 55.94, LDA Noisy Accuracy: 56.06
MLPB ---- Clean Accuracy: 83.69, Noisy Accuracy: 70.49, LDA Clean Accuracy: 78.40, LDA Noisy Accuracy: 66.91
CNN ---- Clean Accuracy: 81.46, Noisy Accuracy: 67.66, LDA Clean Accuracy: 82.51, LDA Noisy Accuracy: 67.54
